##### Лабораторная работа №5 (Проведение исследований с градиентным бустингом)

## Создание бейзлайна и оценка качества

### Обучение моделей из sklearn для выбранных наборов данных

#### Импорт библиотек

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score, log_loss
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from scipy.spatial import distance
import numpy as np
from collections import Counter

#### Подготовка данных

##### Классификация (Rice MSC Dataset)

**Загрузка данных:**

In [18]:
rice_data = pd.read_csv('Rice_MSC_Dataset.csv')

X_rice = rice_data.iloc[:, :-1]  # Признаки
y_rice = rice_data.iloc[:, -1]   # Метки классов

label_encoder = LabelEncoder()
y_rice_encoded = label_encoder.fit_transform(y_rice)

X_rice, _, y_rice_encoded, _ = train_test_split(X_rice, y_rice_encoded, stratify=y_rice_encoded, test_size=0.5, random_state=42)  # уменьшаем датасет для ускорения обучения

**Разделение на тренировочный и тестовый наборы:**

In [19]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)

**Замена пропущенных значений**

In [20]:
def simple_imputer_df(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return pd.DataFrame(imputer.fit_transform(x_train)), pd.DataFrame(imputer.transform(x_test))

In [21]:
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)

**Масштабирование данных**

In [22]:
def scaling(x_train, x_test):
    scaler = StandardScaler()
    return scaler.fit_transform(x_train), scaler.transform(x_test)

In [23]:
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

##### Регрессия (Gold Price Regression)

**Загрузка данных:**

In [24]:
gold_data = pd.read_csv('financial_regression.csv')

X_gold = gold_data.iloc[:, 1:-1]  # Признаки
y_gold = gold_data.iloc[:, -1]   # Целевая переменная

**Разделение на тренировочный и тестовый наборы:**

In [25]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

**Замена пропущенных значений:**

In [26]:
def simple_imputer(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return imputer.fit_transform(x_train), imputer.transform(x_test)

In [27]:
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)

**Удаление строк с пропущенными значениями:**

In [28]:
def delete_none(x_train, y_train, x_test, y_test):
    mask = ~np.isnan(y_train)
    mask_test = ~np.isnan(y_test)
    return x_train[mask], y_train[mask], x_test[mask_test], y_test[mask_test]

In [29]:
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

#### Обучение моделей

##### Классификация

In [30]:
classifier = GradientBoostingClassifier(random_state=42)
classifier.fit(X_train_rice, y_train_rice)

GradientBoostingClassifier(random_state=42)

##### Регрессия

In [31]:
regressor = GradientBoostingRegressor(random_state=42)
regressor.fit(X_train_gold, y_train_gold)

GradientBoostingRegressor(random_state=42)

### Оценка качества моделей по выбранным метрикам

##### Классификация

In [32]:
def quality_eval_classification(X_test, y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    return accuracy, f1

In [33]:
accuracy_origin, f1_origin = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9978666666666667
F1 Score: 0.9978680566773062


##### Регрессия

In [34]:
def quality_eval_regression(X_test, y_test, model):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
    return mae, r2

In [35]:
mae_origin, r2_origin = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2387828.61317904
R-squared: 0.6366372151733224


## Улучшение бейзлайна

### Гипотезы

1. Подбор гиперпараметров:
   - Использование Grid Search с кросс-валидацией для оптимизации модели.

### Проверка гипотез

#### 1. Подбор гиперпараметров

##### Классификация

Подготовка данных:

In [36]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

In [37]:
param_grid = {
    'n_estimators': [100, 150],
    'learning_rate': [0.01, 0.1],
}

Настройка и обучение GridSearchCV:

In [39]:
grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_rice, y_train_rice)

GridSearchCV(estimator=GradientBoostingClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1],
                         'n_estimators': [100, 150]},
             scoring='accuracy')

Вывод лучших параметров и результата:

In [40]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'learning_rate': 0.1, 'n_estimators': 150}
Best cross-validation accuracy:  0.9989000000000001


Оценка качества модели:

In [41]:
quality_eval_classification(X_test_rice, y_test_rice, grid_search.best_estimator_)

Accuracy: 0.9978666666666667
F1 Score: 0.9978680566773062


(0.9978666666666667, 0.9978680566773062)

##### Регрессия

Подготовка данных:

In [42]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

Определение параметров для перебора:

In [43]:
param_grid = {
    'n_estimators': [100, 150],
    'learning_rate': [0.01, 0.1],
}

Настройка и обучение GridSearchCV:

In [44]:
grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_gold, y_train_gold)

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1],
                         'n_estimators': [100, 150]})

Вывод лучших параметров и результата:

In [45]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'learning_rate': 0.1, 'n_estimators': 150}
Best cross-validation accuracy:  0.6687558828807182


Оценка модели с лучшими параметрами на тестовом наборе:

In [46]:
quality_eval_regression(X_test_gold, y_test_gold, grid_search.best_estimator_)

Mean Absolute Error: 2368586.5180117367
R-squared: 0.6477319909538352


(2368586.5180117367, 0.6477319909538352)

### Формирование улучшенного бейзлайна

##### Классификация

Формирование улучшенного бейзлайна:

In [48]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)
                                   
classifier = GradientBoostingClassifier(random_state=42, learning_rate=0.1, n_estimators=150)

##### Регрессия

In [49]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = GradientBoostingRegressor(random_state=42, learning_rate=0.1, n_estimators=150)

### Обучение модели с улучшенным бейзлайном

##### Классификация

Обучение модели:

In [50]:
classifier.fit(X_train_rice, y_train_rice)

GradientBoostingClassifier(n_estimators=150, random_state=42)

##### Регрессия

Обучение модели:

In [51]:
regressor.fit(X_train_gold, y_train_gold)

GradientBoostingRegressor(n_estimators=150, random_state=42)

### Оценка качества модели с улучшенным бейзлайном

##### Классификация

Оценка качества модели:

In [52]:
accuracy_improved, f1_improved = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9978666666666667
F1 Score: 0.9978680566773062


##### Регрессия

Оценка качества модели:

In [53]:
mae_improved, r2_improved = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2368586.5180117367
R-squared: 0.6477319909538352


### Сравнение результатов первоначального бейзлайна с улучшенным бейзлайном

##### Классификация

Видим, что результаты идентичны:

In [54]:
print(f"Accuracy difference: {accuracy_improved-accuracy_origin}")
print(f"F1 Score difference: {f1_improved-f1_origin}")

Accuracy difference: 0.0
F1 Score difference: 0.0


##### Регрессия

Видим, что результаты улучшились:

In [55]:
print(f"Mean Absolute Error difference: {mae_origin-mae_improved}")
print(f"R-squared difference: {r2_improved-r2_origin}")

Mean Absolute Error difference: 19242.095167303458
R-squared difference: 0.011094775780512767


### Выводы

Улучшенный бейзлайн оказался немного лучше базового. Модель классификации вообще не стала лучше -- скорей всего, нужно было ещё поэкспериментировать с гиперпараметрами.

## Имплементация алгоритма машинного обучения

### Самостоятельная имплементация

Для начала разберем, как работает градиентный бустинг:

1. Основная идея: Градиентный бустинг – это метод, который использует ансамбль деревьев решений, где каждое последующее дерево обучается на остатках предыдущих деревьев для исправления ошибок.

2. Пошаговый процесс:
   - Начнем с инициализации модели базовых прогнозов, например, среднего значения для регрессии.
   - Для каждого последующего базового алгоритма мы будем использовать градиент функции потерь для коррекции.
   - Добавляем новое дерево, обученное на этих «остатках», к ансамблю.

##### Классификация

In [107]:
class GradientBoostingMulticlassClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = None
        self.classes_ = None

    def fit(self, X, y):
        n_classes = len(np.unique(y))
        self.classes_ = np.unique(y)
        self.models = {i: [] for i in range(n_classes)}

        y_one_hot = np.zeros((X.shape[0], n_classes))
        y_one_hot[np.arange(X.shape[0]), y] = 1

        logits = np.zeros((X.shape[0], n_classes))

        for _ in range(self.n_estimators):
            probabilities = self._softmax(logits)
            gradients = y_one_hot - probabilities

            for k in range(n_classes):
                # Используем DecisionTreeRegressor вместо DecisionTreeClassifier
                tree = DecisionTreeRegressor(max_depth=self.max_depth)
                tree.fit(X, gradients[:, k])
                update = self.learning_rate * tree.predict(X)
                logits[:, k] += update
                self.models[k].append(tree)

    def _softmax(self, logits):
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        probabilities = exp_logits / exp_logits.sum(axis=1, keepdims=True)
        return probabilities

    def predict(self, X):
        logits = np.zeros((X.shape[0], len(self.classes_)))

        for k, trees in self.models.items():
            for tree in trees:
                update = self.learning_rate * tree.predict(X)
                logits[:, k] += update

        return np.argmax(logits, axis=1)

    def predict_proba(self, X):
        logits = np.zeros((X.shape[0], len(self.classes_)))

        for k, trees in self.models.items():
            for tree in trees:
                update = self.learning_rate * tree.predict(X)
                logits[:, k] += update

        return self._softmax(logits)

##### Регрессия

In [111]:
class SimpleGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        # Инициализация начальных предсказаний (средним значением целевой переменной)
        y_pred = np.full(y.shape, y.mean())

        for _ in range(self.n_estimators):
            # Вычисление остатков
            residuals = y - y_pred

            # Обучение на остатках
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)

            # Обновление предсказаний
            update = self.learning_rate * tree.predict(X)
            y_pred += update

            # Сохранение модели
            self.models.append(tree)

    def predict(self, X):
        # Начальные предсказания (среднее значение целевой переменной)
        y_pred = np.zeros(X.shape[0])

        for tree in self.models:
            # Пошаговое дополнение каждого дерева
            y_pred += self.learning_rate * tree.predict(X)

        return y_pred

### Обучение имплементированной модели

##### Классификация

Подготовка данных:

In [108]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

classifier = GradientBoostingMulticlassClassifier()

Обучение модели:

In [110]:
classifier.fit(X_train_rice, y_train_rice)

##### Регрессия

Подготовка данных:

In [113]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = SimpleGradientBoostingRegressor(max_depth=5)

Обучение модели:

In [114]:
regressor.fit(X_train_gold, y_train_gold)

### Оценка качества имплементированных моделей

##### Классификация

Оценка качества модели:

In [115]:
accuracy_my, f1_my = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9941333333333333
F1 Score: 0.9941383506892516


##### Регрессия

Оценка качества модели:

In [116]:
mae_my, r2_my = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 9639492.618712768
R-squared: -1.343074458974257


### Сравнение результатов первоначального бейзлайна и имплементированных моделей

##### Классификация

Видим, что результаты имплементированной модели немного хуже:

In [117]:
print(f"Accuracy difference: {accuracy_my-accuracy_origin}")
print(f"F1 Score difference: {f1_my-f1_origin}")

Accuracy difference: -0.0037333333333333663
F1 Score difference: -0.0037297059880545547


##### Регрессия

Видим, что результаты имплементированной модели гораздо хуже:

In [118]:
print(f"Mean Absolute Error difference: {mae_my-mae_origin}")
print(f"R-squared difference: {r2_my-r2_origin}")

Mean Absolute Error difference: 7251664.005533728
R-squared difference: -1.9797116741475793


### Выводы

Оценка качества наших имплементированных моделей оказалась хуже.
Можно сделать следующие выводы:
1. Предобработка данных важна: Масштабирование и нормализация данных могут значительно повлиять на качество модели.
2. Оптимизация гиперпараметров: Правильная настройка параметров, таких как темп обучения и количество итераций, необходима для достижения оптимальной производительности.
3. Регуляризация: Помогает избежать переобучения и может улучшить обобщающую способность модели.
4. Алгоритмическая точность: Правильная реализация алгоритма и внимание к численным ошибкам критичны для качества модели.
5. Используйте проверенные решения: Изучение и сравнение с библиотечными реализациями может помочь выявить слабые места вашей модели.
6. Тщательно изучайте данные: Соблюдение чистоты, отсутствие пропусков и ошибок в данных — важный аспект построения качественной модели.

### Добавление техник из улучшенного бейзлайна

##### Классификация

In [121]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

classifier = GradientBoostingMulticlassClassifier(learning_rate=0.1, n_estimators=150)

##### Регрессия

Формирование улучшенного бейзлайна:

In [122]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = SimpleGradientBoostingRegressor(learning_rate=0.1, n_estimators=150)

### Обучение на улучшенном бейзлайне

##### Классификация

In [123]:
classifier.fit(X_train_rice, y_train_rice)

##### Регрессия

In [124]:
regressor.fit(X_train_gold, y_train_gold)

### Оценка качества моделей на улучшенном бейзлайне

##### Классификация

In [126]:
accuracy_my_improved, f1_my_improved = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9957333333333334
F1 Score: 0.9957373023235137


##### Регрессия

In [127]:
mae_my_impoved, r2_my_impoved = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 9664542.658632733
R-squared: -1.347141455293214


### Сравнение результатов

##### Классификация

In [128]:
print(f"Accuracy difference: {accuracy_my_improved-accuracy_improved}")
print(f"F1 Score difference: {f1_my_improved-f1_improved}")

Accuracy difference: -0.0021333333333333204
F1 Score difference: -0.0021307543537925078


##### Регрессия

In [129]:
print(f"Mean Absolute Error difference: {mae_my_impoved-mae_improved}")
print(f"R-squared difference: {r2_my_impoved-r2_improved}")

Mean Absolute Error difference: 7295956.140620996
R-squared difference: -1.9948734462470492


### Выводы

Результаты получились хуже. Наша имплементация для задачи регрессии вообще не сходится. Нужно лучше изучать, как работает градиентный бустинг, и дальше подбирать параметры.

## Подведение итогов / сравнение результатов

Метрика для задачи классификации -- Accuracy

Метрика для задачи регрессии -- R-squared

| Алгоритм            | Задача        | Бейзлайн            | Улучшенный бейзлайн | Самостоятельная имплементация алгоритма |
|---------------------|---------------|---------------------|---------------------|-----------------------------------------|
| KNN                 | классификация | 0.9230666666666667  | 0.9975333333333334  | 0.9978666666666667                      |
|                     | регрессия     | 0.26023132479378674 | 0.5869693562024589  | 0.6048118747567031                      |
| Линейные модели     | классификация | 0.9982              | 0.9986666666666667  | 0.22366666666666668                     |
|                     | регрессия     | 0.6204675812871403  | 0.61988245332444    | 0.5196339914918888                      |
| Решающее дерево     | классификация | 0.9961333333333333  | 0.9958666666666667  | 0.9964                                  |
|                     | регрессия     | 0.1661606200483663  | -1.4486458799625517 | 0.5192060277943523                      |
| Случайный лес       | классификация | 0.9986666666666667  | 0.9987333333333334  | 0.9944666666666667                      |
|                     | регрессия     | 0.610479164922296   | 0.6161405634184933  | 0.5796125932674412                      |
| Градиентный бустинг | классификация | 0.9978666666666667  | 0.9978666666666667  | 0.9957333333333334                      |
|                     | регрессия     | 0.6366372151733224  | 0.6477319909538352  | -1.347141455293214                      |

Лучший алгоритм для задачи классификации -- Случайный лес с улучшенным бейзлайном.

Лучший алгоритм для задачи регрессии -- Градиентный бустинг с улучшенным бейзлайном.